In [1]:
import pandas as pd
import numpy as np
import keras
pd.options.display.max_columns = 999

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../../dsg/demand_anonymized_20170802.csv',sep=';', parse_dates=['Month'])
eval_set = pd.read_csv('../../dsg/eval.csv')
df.head()


,ID,First_MAD,SalOrg,DC,Ship_To,ordre,Plant,Material,ItemCat,OrderQty,LT,LogABC,MOQ,ROP,SafetyStk,PL,MktABC,SubFct,Gross_Weight,Length,Width,Height,Volume,Gamma,Manufacturer,Business,Month,CBO_CBO_Qty_Shortage,Age_ZN_ZI_years,DP_FAMILY_CODE,PRODUCT_STATUS,ORIGINAL_SUPPLIER,SUBRANGE,Comp_reference_number,Name_Of_Competitor,COMP_PRICE_MIN,COMP_PRICE_AVG,COMP_PRICE_MAX,PRICE,NEAREST_COMP_PRICE_MIN,NEAREST_COMP_PRICE_MAX
0,2724174,2014-09-12,yqSu,TUiR,MsHvUF,ggT8Bq,vP6T,iR1zr2,lvOi,1,28.0,Nc4e,72.0,0.0,21.0,NmYB,9rfJ,PjXe,0.006299,0.643863,2.148148,0.693069,0.000480,syni,1nVy21,32af,2014-09-01,NaN,NaN,QIzS,TiQl,X6qzbY,qmsY,1.0,1.0,136.800,136.800000,136.80,152.95,136.80,136.80
1,2325939,2016-02-12,yqSu,csdY,BA5kLC,AIZGVy,vP6T,mD2b7j,lvOi,10,35.0,hfuf,300.0,0.0,87.0,c2Z5,vegq,PjXe,0.000693,0.281690,0.296296,3.613861,0.000151,syni,1nVy21,otZd,2016-02-01,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1836800,2013-05-24,U12J,TUiR,MBZhLQ,8Z4g83,Cisr,AjywTn,lvOi,2,28.0,Nc4e,12.0,0.0,69.0,ss6l,vegq,PjXe,0.038961,1.529175,2.518519,0.653465,0.001261,MYF4,o86vTw,32af,2013-05-01,NaN,2.493151,W2Bc,TiQl,SXlc2j,bUkB,3.0,2.0,411.939,677.394667,830.28,908.60,830.28,830.28
3,2820654,2012-07-12,yqSu,csdY,9Co4iE,RNuiEu,vP6T,DPiXfc,lvOi,10,35.0,Nc4e,450.0,0.0,148.0,c2Z5,vegq,PjXe,0.000682,0.241449,4.814815,0.247525,0.000144,syni,1nVy21,otZd,2012-07-01,NaN,NaN,zAA4,TiQl,Rmt5iw,5cy5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5579286,2012-02-29,OQfZ,TUiR,88lPvN,KPTgL7,9O7Z,tOsuQV,lvOi,6,28.0,Nc4e,460.0,0.0,500.0,c2Z5,9rfJ,PjXe,0.000693,0.261569,4.481481,0.173267,0.000100,9AXw,OEScPd,otZd,2012-02-01,NaN,NaN,yZYX,TiQl,ikezPZ,1ftA,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
series = df.groupby(["SalOrg", "Material", "Month"])["OrderQty"].sum().reset_index()
series = series.sort_values(by=["SalOrg", "Material", "Month"])
series.head()

,SalOrg,Material,Month,OrderQty
0,97LK,00IYcj,2012-05-01,2
1,97LK,00IYcj,2012-06-01,13
2,97LK,00IYcj,2012-07-01,1
3,97LK,00IYcj,2012-09-01,30
4,97LK,00IYcj,2012-11-01,1


In [4]:
import itertools
# fill empty
eval_comb = eval_set[['Material', 'SalOrg']]
eval_comb = list(set([tuple(x) for x in eval_comb.values]))

comb = list(itertools.product(*[eval_comb, list(series['Month'].unique())]))
comb = [(t[0], t[1], m) for t, m in comb]

series2 = pd.DataFrame(comb, columns=['Material', 'SalOrg', 'Month'])
series2 = series2.sort_values(by=['Material', 'SalOrg', 'Month' ])
series2 = series2.merge(series, on=['Month', 'Material', 'SalOrg'], how='left')
series2 = series2.fillna(0)

In [5]:
from dateutil.relativedelta import relativedelta

# validation folds
def create_validation(data, date, m):
    return data[(data["Month"] > pd.to_datetime(date) - relativedelta(months=m))&
                (data["Month"] < pd.to_datetime(date))].index, data[(data["Month"] >= pd.to_datetime(date)) & 
                (data["Month"] < pd.to_datetime(date) + relativedelta(months=3))].index

validation_months = ['2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01']

folds = []
for month in validation_months:
    folds.append(create_validation(series2, month, 4))

In [6]:
series2.head()

,Material,SalOrg,Month,OrderQty
0,00GB1f,yqSu,2012-01-01,0.0
1,00GB1f,yqSu,2012-02-01,0.0
2,00GB1f,yqSu,2012-03-01,1.0
3,00GB1f,yqSu,2012-04-01,3.0
4,00GB1f,yqSu,2012-05-01,2.0


In [7]:
for fold in folds:
    print("Train:", str(series2.loc[fold[0], "Month"].min())[:10], "–", str(series2.loc[fold[0], "Month"].max())[:10],
          "    Validation:", str(series2.loc[fold[1], "Month"].min())[:10], "–", str(series2.loc[fold[1], "Month"].max())[:10])

Train: 2016-05-01 – 2016-07-01     Validation: 2016-08-01 – 2016-10-01
Train: 2016-06-01 – 2016-08-01     Validation: 2016-09-01 – 2016-11-01
Train: 2016-07-01 – 2016-09-01     Validation: 2016-10-01 – 2016-12-01
Train: 2016-08-01 – 2016-10-01     Validation: 2016-11-01 – 2017-01-01


In [8]:
series2.head()

,Material,SalOrg,Month,OrderQty
0,00GB1f,yqSu,2012-01-01,0.0
1,00GB1f,yqSu,2012-02-01,0.0
2,00GB1f,yqSu,2012-03-01,1.0
3,00GB1f,yqSu,2012-04-01,3.0
4,00GB1f,yqSu,2012-05-01,2.0


In [9]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, SimpleRNN, GRU

def lag_feature(df, colname, lag, adv):
    cols_lagged = []
    for i in range(lag, 0, -1):
        col_lagged = '{}(t-{})'.format(colname, i)
        df[col_lagged] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(i)
        cols_lagged.append(col_lagged)
    cols_adv = []
    for i in range(1, adv):
        col_adv = '{}(t+{})'.format(colname, i)
        df[col_adv] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(-i)
        cols_adv.append(col_adv) 
        
    return df, cols_lagged, cols_adv

series, lagged, adv = lag_feature(series2, 'OrderQty', 12, 3)
series.head()

,Material,SalOrg,Month,OrderQty,OrderQty(t-12),OrderQty(t-11),OrderQty(t-10),OrderQty(t-9),OrderQty(t-8),OrderQty(t-7),OrderQty(t-6),OrderQty(t-5),OrderQty(t-4),OrderQty(t-3),OrderQty(t-2),OrderQty(t-1),OrderQty(t+1),OrderQty(t+2)
0,00GB1f,yqSu,2012-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
1,00GB1f,yqSu,2012-02-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,3.0
2,00GB1f,yqSu,2012-03-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,3.0,2.0
3,00GB1f,yqSu,2012-04-01,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,2.0,1.0
4,00GB1f,yqSu,2012-05-01,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,3.0,1.0,3.0


In [ ]:
# load dataset
reframed = series[lagged]
reframed.dropna(inplace=True)
idx = reframed.index
values = reframed.values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = StandardScaler()
scaler = scaler.fit(values)


In [ ]:
# split into input and outputs
for fold in folds: 
    reframed.index = idx
    train = reframed.loc[fold[0]]
    test = reframed.loc[fold[1]]
    scaled_train = scaler.transform(train)
    scaled_val = scaler.transform(test)

    train_X, train_y = scaled_train[:, :-3], scaled_train[:, -3:]
    test_X, test_y = scaled_val[:, :-3], scaled_val[:, -3:]
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], 1))
    test_X = test_X.reshape((test_X.shape[0],test_X.shape[1], 1))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    model = Sequential()
    model.add(GRU(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(3))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    history = model.fit(train_X, train_y, epochs=50, batch_size=128, validation_data=(test_X, test_y), verbose=1, shuffle=False)

In [ ]:
train_Y

In [ ]:
# design network
model = Sequential()
model.add(GRU(50, input_shape=(train_X.shape[1], train_X.shape[2])),return_sequences=False)
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=256, verbose=1, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
ls

In [ ]:
reframed.index = idx
train = reframed.loc[fold[0]]
test = reframed.loc[fold[1]]
scaled_train = scaler.transform(train)
scaled_val = scaler.transform(test)

train_X, train_y = scaled_train[:, :-2], scaled_train[:, -2:]
test_X, test_y = scaled_val[:, :-2], scaled_val[:, -2:]

In [ ]:
test_X.shape

In [ ]:
test_y.shape

In [ ]:
np.concatenate((test_X,test_y), axis=1)

In [ ]:
scaled_val

In [ ]:
reframed

In [ ]:
test_i = series[series["Month"]==pd.to_datetime('2017-03-01')].index

test_x = series.loc[test_i][lagged]
test_x